In [1]:
import json
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [2]:
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import GridappsdConnection
from cimgraph.models import FeederModel
# from cimgraph.models import DistributedArea

In [3]:
import cimgraph.utils as utils

In [4]:
params = ConnectionParameters(host = "localhost", port = "61613", cim_profile='rc4_2021', iec61970_301=7)
gapps = GridappsdConnection(params)

In [5]:
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=gapps, container=feeder, distributed=False)


In [6]:
# feeder_mrids = ["_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62", # 13 bus
#                 "_C1C3E687-6FFD-C753-582B-632A27E28507",  # 123 bus
#                 "5B816B93-7A5F-B64C-8460-47C17D6E4B0F", # 13 bus asets
#                 "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3",  # 8500 node
#                 "_67AB291F-DCCD-31B7-B499-338206B9828F", # J1
#                 "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"]  # R2 12.47 3
# # feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node
# # feeder_mrid = "_9E985101-27AD-4FE4-B36A-EBECC98CDFAB" #MAPLE 10 bus
# for feeder_mrid in feeder_mrids:
#     feeder = cim.Feeder(mRID = feeder_mrid)
#     network = FeederModel(connection=gapps, container=feeder, distributed=False)
#     test_get_all_edges(network, cim)

In [7]:
with open("topo_message.json") as topo:
    topo_msg = json.load(topo)

In [8]:
# central_network = FeederModel(connection=gapps, container=feeder, distributed=False)

# dist_network_2 =  FeederModel(connection=gapps, container=feeder, distributed=True)

# dist_network_3 =  FeederModel(connection=gapps, container=feeder, distributed=True, topology_message=topo_msg)

In [9]:
# for switch_area in dist_network_2.distributed_areas:
#     test_get_all_edges(switch_area, cim)
#     for sec_area in switch_area.distributed_areas:
#         test_get_all_edges(sec_area, cim)

In [10]:
# for switch_area in dist_network_3.distributed_areas:
#     test_get_all_edges(switch_area, cim)
#     for sec_area in switch_area.distributed_areas:
#         test_get_all_edges(sec_area, cim)

In [11]:
# from cimgraph.models import DistributedArea

# container = cim.EquipmentContainer(mRID=feeder_mrid)
# feeder_area = DistributedArea(container, gapps, distributed=True)
# feeder_area.build_from_topo_message(topo_msg['feeders'])

# feeder_area.get_all_edges(cim.ConnectivityNode)
# for node in feeder_area.graph[cim.ConnectivityNode].values():
#     print(node.name)

In [12]:
# counter = 0
# for switch_topo in topo_msg['feeders']['switch_areas']:
#     print('switch area', counter)
#     container = cim.EquipmentContainer(mRID=feeder_mrid+'.'+str(counter))
#     SwitchArea = DistributedArea(container, gapps, distributed=True)
#     SwitchArea.build_from_topo_message(switch_topo)
#     counter = counter + 1

#     SwitchArea.get_all_edges(cim.ConnectivityNode)
#     for node in SwitchArea.graph[cim.ConnectivityNode].values():
#         print(node.name)

In [13]:
# sw_counter = -1
# for switch_topo in topo_msg['feeders']['switch_areas']:
#     sw_counter +=1
#     sa_counter = -1
#     for sec_topo in switch_topo['secondary_areas']:
#         sa_counter +=1
#         area_id = str(sw_counter) + '.' + str(sa_counter)
#         print('sec area', area_id)
#         container = cim.EquipmentContainer(mRID=area_id)
#         SecondaryArea = DistributedArea(container, gapps, distributed=True)
#         SecondaryArea.build_from_topo_message(sec_topo)

#         SecondaryArea.get_all_edges(cim.ConnectivityNode)
#         for node in SecondaryArea.graph[cim.ConnectivityNode].values():
#             print(node.name)

In [14]:
utils.get_all_line_data(network)
for phs in network.graph[cim.ACLineSegmentPhase].values():
    print(type(phs.phase))

<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>
<enum 'SinglePhaseKind'>


In [15]:
network.get_all_edges(cim.ACLineSegment)

In [16]:
query = (network.get_edges_query(cim.ACLineSegmentPhase))
print(query)


        PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX cim:  <http://iec.ch/TC57/CIM100#>
        SELECT DISTINCT ?mRID ?attribute ?value ?edge
        WHERE {
          ?eq r:type cim:ACLineSegmentPhase.
        VALUES ?mRID { "_94FCAFFF-E5F7-4F97-B697-35C46656EDCA" 
 "_CE2AFB05-A1B1-4082-B4EB-2AD8A1D3422D" 
 "_D9C9CEA6-B3F0-4E02-9911-3DFA192B4D01" 
 "_19B6044B-6915-4245-9DD1-6AB0444D42A6" 
 "_E295A13C-1B49-4D5A-8BB6-6D106AE0D436" 
 "_5092EC91-3F74-4829-ABF1-AF5C6AFD1A50" 
 "_5AFA6BC8-9FCE-4931-A83B-C8CA2EC15551" 
 "_1B6C674F-B22C-4D39-9294-98BB4634ED96" 
 "_A28877AB-3D74-4950-AA8C-45D962BD7331" 
 "_A539F769-AF0F-4203-99CE-6FF4B9D07F70" 
 "_216C7BCA-22B0-4B68-81F5-33AB8A4E6DA7" 
 "_5D4FC857-8FD9-4BE9-8C1F-11CF7E38AA46" 
 "_BD4212B3-1635-421A-BDD8-767B6738FA80" 
 "_AA2FE477-C2D1-4206-9A8E-742D27976105" 
 "_2A5A1025-AD81-4BFA-A7F0-ED70F7F43DAA" 
 "_2D60B241-DE87-48BD-A8D3-FE4B705B40ED" 
 "_A66508A9-C9C1-4DB5-8FFD-F081E5E9E14D" 
 "_28DAC6BE-8DD5-4C61-958E-ADB4E8